In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#         break

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import All Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score , auc
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_hub as hub 

from keras.applications.vgg19 import VGG19

In [ ]:

train_csv_path = "/kaggle/input/UBC-OCEAN/train.csv"
test_csv_path = "/kaggle/input/UBC-OCEAN/test.csv"

train_df = pd.read_csv(train_csv_path)
train_df

In [ ]:
sns.countplot(x=train_df['label'])
plt.show()

In [ ]:

plt.scatter(train_df['image_width'],train_df['image_height'])
plt.show()

In [ ]:
train_df['is_tma'] = train_df['is_tma'].astype('int8')

In [ ]:
train_df['is_tma'].value_counts().plot(kind="pie",autopct="%.1f%%")
plt.title("Image Distributions on Train Data")
plt.legend()
plt.show()

In [ ]:
train_thumbnails = os.listdir("/kaggle/input/UBC-OCEAN/train_thumbnails")
train_images = os.listdir("/kaggle/input/UBC-OCEAN/train_images")

In [ ]:
class_labels = ['CC', 'EC', 'HGSC', 'LGSC', 'MC']
train_df['label'] = train_df['label'].replace({'CC':0, 'EC':1, 'HGSC':2, 'LGSC':3, 'MC':4})
train_df.head()

In [ ]:
train_df['image_id']=train_df['image_id'].astype("int32")
train_df['label']=train_df['label'].astype("int8")
train_df['image_width']=train_df['image_width'].astype("int32")
train_df['image_height']=train_df['image_height'].astype("int32")

In [ ]:
#train_df['label'].value_counts()
train_df.info()

In [ ]:
train_df.head()

## Training Images Preprocessing

In [ ]:
Image.MAX_IMAGE_PIXELS = 10000000000
# Define patch size and overlap (if needed)
patch_size = (128,128)  # Adjust this according to your requirements
overlap = 10  # Adjust this if you want overlapping patches

image_data = []
image_label = []
empty_img=0
for img_id, label , tma in zip(train_df['image_id'],train_df['label'], train_df['is_tma']):
    #print(img_id, label,  tma)
    if tma==0:
        img_name = str(img_id)+"_thumbnail.png"
        large_image = Image.open("/kaggle/input/UBC-OCEAN/train_thumbnails/"+img_name)
        for y in range(0, large_image.height, patch_size[0] - overlap): # (0,2523,192)
            for x in range(0, large_image.width, patch_size[1] - overlap):  # (0,3000,192)  224-32=192
                patch = large_image.crop((x, y, x+patch_size[1], y+patch_size[0]))
                image = np.array(patch)
                if np.sum(image)==0:
                    empty_img+=1
                elif (np.sum(image[0:,0:50])==0) or (np.sum(image[0:,50:])==0) or (np.sum(image[0:,100:])==0):
                    empty_img+=1
                elif (np.sum(image[0:50,0:])==0) or (np.sum(image[50:,0:])==0) or (np.sum(image[100:,0:])==0):
                    empty_img+=1
                elif (np.sum(image[0:50,0:50])==0) or (np.sum(image[0:50,50:])==0):
                    empty_img+=1
                elif (np.sum(image[50:100,0:50])==0) or (np.sum(image[50:100,50:])==0):
                    empty_img+=1
                elif (np.sum(image[50:,0:100])==0) or (np.sum(image[80:,80:])==0):
                    empty_img+=1
                elif (np.sum(image[0:50,75:])==0) or (np.sum(image[50:,75:])==0):
                    empty_img+=1
                elif (np.sum(image[0:40,80:])==0) or (np.sum(image[80:,80:])==0):
                    empty_img+=1
                elif (np.sum(image[0:35,0:])==0) or (np.sum(image[80:,0:])==0):
                    empty_img+=1
                elif (np.sum(image[0:25,0:50])==0) or (np.sum(image[0:25,100:])==0):
                    empty_img+=1
                elif (np.sum(image[0:20,0:25])==0) or (np.sum(image[0:20,25:50])==0) or (np.sum(image[0:20,50:80])==0) or (np.sum(image[0:20,90:])==0):
                    empty_img+=1
                elif (np.sum(image[0:25,0:25])==0) or (np.sum(image[0:25,100:])==0) or (np.sum(image[100:,0:25])==0) or (np.sum(image[100:,100:])==0):
                    empty_img+=1
                elif (np.sum(image[0:15,0:])==0) or (np.sum(image[115:,0:])==0):
                    empty_img+=1
                elif (np.sum(image[0:,0:15])==0) or (np.sum(image[0:,115:])==0):
                    empty_img+=1
                elif (np.sum(image[0:20,0:20])==0) or (np.sum(image[0:20,110:])==0) or (np.sum(image[110:,0:20])==0) or (np.sum(image[110:,110:])==0):
                    empty_img+=1
                elif (np.sum(image[0:10,0:10])==0) or (np.sum(image[0:10,115:])==0) or (np.sum(image[0:10,40:60])==0) or (np.sum(image[0:10,80:100])==0):
                    empty_img+=1
                elif (np.sum(image[40:50,0:10])==0) or (np.sum(image[110:,80:100])==0) or (np.sum(image[70:85,70:90])==0) or (np.sum(image[50:60,110:])==0):
                    empty_img+=1

                else:
                    image_data.append(image)
                    image_label.append(label)
        
#     if tma==1:
#         img_name = str(img_id)+".png"
#         large_image = Image.open("/kaggle/input/UBC-OCEAN/train_images/"+img_name)
#         for y in range(0, large_image.height, patch_size[0] - overlap): # (0,2523,192)
#             for x in range(0, large_image.width, patch_size[1] - overlap):  # (0,3000,192)  224-32=192
#                 patch = large_image.crop((x, y, x+patch_size[1], y+patch_size[0]))
#                 image = np.array(patch)
#                 if np.sum(image)==0:
#                     empty_img+=1
#                 elif (np.sum(image[0:,0:50])==0) or (np.sum(image[0:,50:])==0) or (np.sum(image[0:,100:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:50,0:])==0) or (np.sum(image[50:,0:])==0) or (np.sum(image[100:,0:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:50,0:50])==0) or (np.sum(image[0:50,50:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[50:100,0:50])==0) or (np.sum(image[50:100,50:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[50:,0:100])==0) or (np.sum(image[80:,80:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:50,75:])==0) or (np.sum(image[50:,75:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:40,80:])==0) or (np.sum(image[80:,80:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:35,0:])==0) or (np.sum(image[80:,0:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:25,0:50])==0) or (np.sum(image[0:25,100:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:20,0:25])==0) or (np.sum(image[0:20,25:50])==0) or (np.sum(image[0:20,50:80])==0) or (np.sum(image[0:20,90:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:25,0:25])==0) or (np.sum(image[0:25,100:])==0) or (np.sum(image[100:,0:25])==0) or (np.sum(image[100:,100:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:15,0:])==0) or (np.sum(image[115:,0:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:,0:15])==0) or (np.sum(image[0:,115:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:20,0:20])==0) or (np.sum(image[0:20,110:])==0) or (np.sum(image[110:,0:20])==0) or (np.sum(image[110:,110:])==0):
#                     empty_img+=1
#                 elif (np.sum(image[0:10,0:10])==0) or (np.sum(image[0:10,115:])==0) or (np.sum(image[0:10,40:60])==0) or (np.sum(image[0:10,80:100])==0):
#                     empty_img+=1
#                 elif (np.sum(image[40:50,0:10])==0) or (np.sum(image[110:,80:100])==0) or (np.sum(image[70:85,70:90])==0) or (np.sum(image[50:60,110:])==0):
#                     empty_img+=1
                
                    
#                 else:
#                     image_data.append(image)
#                     image_label.append(label)

In [ ]:

print(len(image_data))
print(len(image_label))
print(empty_img)

In [ ]:
image_data[0].shape

## Training Images Visualization

In [ ]:
set(image_label)

## Covert image data into array for training

In [ ]:
x = np.array(image_data) 
y = np.array(image_label)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
# from sklearn.utils import shuffle

In [ ]:
# x,y=shuffle(x,y,random_state=42)

## Split The Data

In [ ]:
x = x[:5000]
y = y[:5000]

In [ ]:
from sklearn.utils import shuffle
X_train,y_train=shuffle(x,y, random_state=452)

In [ ]:
x_train, x_test ,y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=True)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
yest=y_test
yest.shape


## Scale The Data for Train images

In [ ]:
x_train_scaled = x_train/255
x_test_scaled = x_test/255

## Train Image Visualization

## Test Image Visualization

## Model Building Using VGG16 Model

In [ ]:
# from tensorflow.keras.applications import VGG16
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.optimizers import Adam
# import tensorflow as tf
# num_classes = 5
# # Load the VGG16 model with ImageNet weights and exclude the top classification layer
# base_model =VGG16(weights='imagenet', include_top=False, input_shape=(128,128, 3))

# # Customize the top classification layers
# x = base_model.output
# x = Flatten()(x)
# x = Dense(4096, activation='relu')(x)
# x = Dense(4096, activation='relu')(x)
# predictions = Dense(num_classes, activation='softmax')(x)  # Replace num_classes with the number of classes in your dataset

# # Create the VGG16 model with your custom top layer
# model = Model(inputs=base_model.input, outputs=predictions)

# # Freeze pre-trained layers (optional)
# for layer in base_model.layers:
#     layer.trainable = False



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, Dropout, Dense, Multiply, Add
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

# Encoder block
def encoder_block(inputs, filters, kernel_size, activation='relu'):
    x = SeparableConv2D(filters, kernel_size, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = SeparableConv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

# Decoder block with attention
def decoder_block(inputs, skip_features, filters, kernel_size, activation='relu'):
    x = SeparableConv2D(filters, kernel_size, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = SeparableConv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Add()([x, skip_features])
    return x

# Attention block
def attention_block(inputs, skip_features):
    g = Conv2D(filters=skip_features.shape[-1], kernel_size=(1, 1), strides=(1, 1), padding='same')(inputs)
    x = Conv2D(filters=skip_features.shape[-1], kernel_size=(1, 1), strides=(1, 1), padding='same')(skip_features)
    attn = Multiply()([g, x])
    
    # Use a 1x1 convolution to match shapes if needed
    attn = Conv2D(filters=inputs.shape[-1], kernel_size=(1, 1), strides=(1, 1), padding='same')(attn)
    
    x = Add()([inputs, attn])
    return x

# Input layer
input_layer = Input(shape=(128, 128, 3))

# Encoder
encoder1 = encoder_block(input_layer, 64, 3)
encoder2 = encoder_block(encoder1, 128, 3)
encoder3 = encoder_block(encoder2, 256, 3)

# Attention
attention = attention_block(encoder3, encoder2)

# Decoder
decoder1 = decoder_block(attention, encoder2, 128, 3)
decoder2 = decoder_block(decoder1, encoder1, 64, 3)
# Global Average Pooling
x = GlobalAveragePooling2D()(decoder2)

# Fully connected layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(5, activation='softmax')(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Display model summary
model.summary()
model.compile(optimizer=tf.keras.optimizers.Nadam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Nadam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model on your new dataset
history = model.fit(x_train_scaled, y_train, epochs=20, batch_size=16,
                   validation_data=(x_test_scaled,y_test))

In [ ]:
class_labels = ['CC','EC', 'HGSC', 'LGSC','MC']
plt.figure(figsize=(16,8))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual Label:{class_labels[y_test[i]]}\nPredicted Label:{class_labels[y_pred_test[i]]}")
    plt.axis("off")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b',linestyle = 'dashdot', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', linestyle = 'dashdot',label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b',linestyle = 'dashdot' ,label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',linestyle = 'dashdot',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
# Turn off the grid
ax[0].grid(False)
ax[1].grid(False)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Assuming 'model' is your Keras functional model
x_train_each_class = ['CC','EC', 'HGSC', 'LGSC','MC']

# Convert multiclass labels to one-hot encoding
y_test_bin = label_binarize(y_test, classes=np.arange(4))

# Get predicted probabilities for each class
y_pred_proba = model.predict(x_test_scaled)

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves for each class
plt.figure(figsize=(8, 6))
for i in range(4):
    plt.plot(fpr[i], tpr[i], label=f'Class {x_train_each_class[i]} (AUC= {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.show()


## Model Evaluation on Train & Test Data

In [ ]:
loss ,acc = model.evaluate(x_train_scaled, y_train)
print("Accuracy on Train Data:",acc)
print()
loss ,acc = model.evaluate(x_test_scaled, y_test )
print("Accuracy on Test Data:",acc)

In [ ]:
Y_pred=x.predict(X_test)
Y_pred[:5]

In [ ]:
y_pred = model.predict(x_test_scaled)
y_pred_test = [np.argmax(i) for i in y_pred]

In [ ]:
#len(y_pred_test)

In [ ]:
# Save the model to a file
# mobilenet_model.save("mobilenet_model_3.h5")

## Metrics Evaluation on Test Data

In [ ]:
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred_test))
print()


print("Classification Report:\n",classification_report(y_test,y_pred_test))

In [ ]:
labels=['CC','EC', 'HGSC', 'LGSC','MC']

In [ ]:
cm_data=confusion_matrix(y_test,y_pred_test)
cm = pd.DataFrame(cm_data, columns=labels, index =labels)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (5,5))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="crest", annot=True, annot_kws={"size": 16}, fmt='g')

## Compare Actual & Predicted Labels

In [ ]:
plt.figure(figsize=(24,40))
for i in range(60):
    plt.subplot(10,6,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual Label:{class_labels[y_test[i]]}\nPredicted Label:{class_labels[y_pred_test[i]]}")
    plt.axis("off")

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# Lists to store accuracy values for each fold
train_accuracies = []
val_accuracies = []

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X_train, y_train):
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(X_train[train], y_train[train], validation_data=(X_train[test], y_train[test]),
                        epochs=10, verbose=0, batch_size=8)

    # Calculate and print the training accuracy
    train_loss, train_accuracy = model.evaluate(X_train[train], y_train[train], verbose=0)
    print(f'Fold {fold_no} Training Loss: {train_loss}')
    print(f'Fold {fold_no} Training Accuracy: {train_accuracy}')
    train_accuracies.append(train_accuracy)

    # Calculate and print the validation accuracy
    val_loss, val_accuracy =model.evaluate(X_train[test], y_train[test], verbose=0)
    print(f'Fold {fold_no} Validation Loss: {val_loss}')
    print(f'Fold {fold_no} Validation Accuracy: {val_accuracy}')
    val_accuracies.append(val_accuracy)

    # Increase fold number
    fold_no += 1